In [1]:
import sys
from outletsBiasRatings import outletsBiasRatingsAllSides,outletAbbreviationToFullName
from matplotlib.pyplot import *
import pandas as pd
import pickle
import time
import os
from joblib import Parallel, delayed
from transformers import pipeline, AutoTokenizer, AutoModelWithLMHead

In [2]:
basePath = 'D:/media'
outletsBiasRatings = outletsBiasRatingsAllSides
outlets = list(outletsBiasRatings.keys())
# BATCH_SIZE=64
BATCH_SIZE=128
HEADLINE_TOKENS_MAX_LENGTH=32
print(len(outlets))
print(outlets)

47
['alternet', 'democracynow', 'db', 'hp', 'theintercept', 'jacobin', 'motherjones', 'thenewyorker', 'thenation', 'slate', 'vox', 'cnn', 'nyt', 'abcnews', 'theatlantic', 'buzzfeed', 'cbs', 'economist', 'guardian', 'nbcnews', 'politico', 'timemagazine', 'wp', 'npr', 'ap', 'bbc', 'bloomberg', 'csm', 'reuters', 'thehill', 'usatoday', 'wsj', 'reason', 'we', 'wt', 'fox', 'americanspectator', 'bre', 'theblaze', 'cbn', 'dailycaller', 'dailymail', 'dailywire', 'thefederalist', 'nationalreview', 'nyp', 'newsmax']


## Downloading and saving default sentiment analysis pipeline from HuggingFace

In [3]:
# # # from transformers import pipeline

# # nlp = pipeline("sentiment-analysis")#default distilbert-base-uncased-finetuned-sst-2-english
# nlp = pipeline("sentiment-analysis",model="siebert/sentiment-roberta-large-english")

# result = nlp("I hate you")[0]
# print(f"label: {result['label']}, with score: {round(result['score'], 4)}")
      
# # nlp.save_pretrained("./distilbert-base-uncased-finetuned-sst-2-english")
# nlp.save_pretrained("./siebert/sentiment-roberta-large-english")

## Loading sentiment analysis pipeline from local store

In [4]:
# sentimentPipeline = pipeline(task='sentiment-analysis',model="./distilbert-base-uncased-finetuned-sst-2-english",
#                tokenizer="./distilbert-base-uncased-finetuned-sst-2-english",device=0)#gpu

sentimentPipeline = pipeline("sentiment-analysis",model="./siebert/sentiment-roberta-large-english",device=0)

Some weights of RobertaModel were not initialized from the model checkpoint at ./siebert/sentiment-roberta-large-english and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
def get_sentiment(sentimentPipeline,batch):
    labelsToNumbersDict={ 
                         'NEGATIVE':-1,'POSITIVE':1,
                        }
    try:
        predictions=sentimentPipeline(batch)
        numericPredictions=[]
        for prediction in predictions:
#             {'label': 'NEGATIVE', 'score': 0.952504575252533}
            #weighted or not weighted but very similar results regardless:
#             numericPrediction=labelsToNumbersDict[prediction['label']]
            numericPrediction=labelsToNumbersDict[prediction['label']] * prediction['score'] #Weight the sentiment prediction
    
            numericPredictions.append(numericPrediction)
        return numericPredictions
    except Exception as e:
#         raise
        print(e)  
        return None

In [6]:
#testing
batch=[
     "You are a piece of shit","Looking forward to meeting you","Canada: Buyout Challenged Over Racial Bias",
      "New H.I.V. Cases Rise in Young Gay Men",
      "Death Toll From Syria's Increasingly Grim War Passes 60,000",
    "Donald Trump’s re-election campaign poses the greatest threat to American democracy since World War II.",
    "Erin Wall, 44; Acclaimed Soprano in Mozart and Strauss",
    "Drug May Extend A.L.S. Patients’ Lives by Several Months, Study Finds",
      ]
print(sentimentPipeline(batch))
print(get_sentiment(sentimentPipeline,batch))

[{'label': 'NEGATIVE', 'score': 0.9993973970413208}, {'label': 'POSITIVE', 'score': 0.9988988637924194}, {'label': 'NEGATIVE', 'score': 0.9986793994903564}, {'label': 'NEGATIVE', 'score': 0.9823978543281555}, {'label': 'NEGATIVE', 'score': 0.9927184581756592}, {'label': 'NEGATIVE', 'score': 0.9969961643218994}, {'label': 'POSITIVE', 'score': 0.9983787536621094}, {'label': 'POSITIVE', 'score': 0.9964010715484619}]
[-0.9993973970413208, 0.9988988637924194, -0.9986793994903564, -0.9823978543281555, -0.9927184581756592, -0.9969961643218994, 0.9983787536621094, 0.9964010715484619]


In [7]:
for t in zip(batch,get_sentiment(sentimentPipeline,batch)):
    print(t[0],t[1])

You are a piece of shit -0.9993973970413208
Looking forward to meeting you 0.9988988637924194
Canada: Buyout Challenged Over Racial Bias -0.9986793994903564
New H.I.V. Cases Rise in Young Gay Men -0.9823978543281555
Death Toll From Syria's Increasingly Grim War Passes 60,000 -0.9927184581756592
Donald Trump’s re-election campaign poses the greatest threat to American democracy since World War II. -0.9969961643218994
Erin Wall, 44; Acclaimed Soprano in Mozart and Strauss 0.9983787536621094
Drug May Extend A.L.S. Patients’ Lives by Several Months, Study Finds 0.9964010715484619


In [8]:
def annotateSentimentInOutletHeadlines(outlet,startYear,endYear):
    print(outlet,end=',')
    inputPath=rf'.\headlinesData\{outlet}'
#     outputPath=rf'.\headlinesDataWithSentimentLabelsAnnotationsFromDistilbertSST2\{outlet}'
    outputPath=rf'.\headlinesDataWithSentimentLabelsAnnotationsFromSentimentRobertaLargeModel\{outlet}'
    if not os.path.exists(outputPath):
        os.makedirs(outputPath)
    files=os.listdir(inputPath)
    for file in files:
        year=file.split('.')[0]
        headlinesAll=[]
        urlsAll=[]
        sentimentLabelsAll=[]
        if startYear <= int(year) <= endYear:
            print(year,end=',')
            inputFilePath=os.path.join(inputPath,file)
            df=pd.read_csv(inputFilePath)
            for rowIndex in range(0,len(df),BATCH_SIZE):
                try:
                    dfBatch=df[rowIndex:rowIndex+BATCH_SIZE]
                    headlines=list(dfBatch['title'].values)
                    urls=list(dfBatch['url'].values)
                    
                    headlinesToUrlsDict={}
                    for headlineIndex, headline in enumerate(headlines):
                        headlinesToUrlsDict[headline]=urls[headlineIndex]
                        
                    headlinesBatch=[headline for headline in headlines if (type(headline)==str and len(headline.split())<=HEADLINE_TOKENS_MAX_LENGTH)]
                    
                    sentimentLabels=get_sentiment(sentimentPipeline,headlinesBatch)
                    sentimentLabelsAll += sentimentLabels
                    
                    headlinesAll += headlinesBatch
                    
                    urlsInBatch = [headlinesToUrlsDict[headline] for headline in headlinesBatch]
                    urlsAll += urlsInBatch
                    
                except Exception as e:
#                     raise
                    print("Something went wrong in sentence labeling loop", e)
                    continue
            dfOut = pd.DataFrame(list(zip(sentimentLabelsAll,headlinesAll,urlsAll)), columns =['sentiment', 'headline','url']) 
            outputFilePath=os.path.join(outputPath,year+'.csv')
            dfOut.to_csv(outputFilePath,index=False)
    print()

In [9]:
startYear = 2000
endYear = 2019
outlets = list(outletsBiasRatings.keys())
for outlet in outlets:
    annotateSentimentInOutletHeadlines(outlet,startYear,endYear)

alternet,2012,2013,2014,2015,2016,2017,2018,2019,
democracynow,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,
db,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,
hp,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,
theintercept,2014,2015,2016,2017,2018,2019,
jacobin,2011,2012,2013,2014,2015,2016,2017,2018,2019,
motherjones,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,
thenewyorker,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,
thenation,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,
slate,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,
vox,2014,2015,2016,2017,2018,2019,
cnn,2011,2012,2013,2014,2015,2016,2017,2018,2019,
nyt,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2